# [Day 10](https://adventofcode.com/2023/day/10)
## Part 1

In [1]:
from collections import defaultdict
from dataclasses import dataclass
from typing import Iterable, TypedDict, NewType


Node = NewType('Node', tuple[int, int])
Tube = NewType('Tube', tuple[Node, str])

@dataclass(frozen=True)
class Edge:
    tube: Tube
    node: Node


def readInput(filename) -> (set[Edge], dict[Node, set[Edge]], dict[Tube, str]):
    """
    Reads the input file and returns the start nodes, the edges and the map of tubes
    """

    tubesMap: dict[Node, str] = dict()
    edges: dict[Node, set[Edge]] = defaultdict(set[Edge])
    start: Tube = None

    with open(filename, 'r') as f:
        y = 0
        for line in f:
            line = line.strip()

            x = 0
            for c in line:
                tube: Tube = None
                if c != '.':
                    tube = ((y, x), c)
                    tubesMap[(y, x)] = c
                match c:
                    case 'S':
                        start = ((y, x), c)
                    case '|':
                        edges[(y+1, x)].add(Edge(tube, (y-1, x)))
                        edges[(y-1, x)].add(Edge(tube, (y+1, x)))
                    case '-':
                        edges[(y, x+1)].add(Edge(tube, (y, x-1)))
                        edges[(y, x-1)].add(Edge(tube, (y, x+1)))
                    case 'L':
                        edges[(y-1, x)].add(Edge(tube, (y, x+1)))
                        edges[(y, x+1)].add(Edge(tube, (y-1, x)))
                    case 'J':
                        edges[(y-1, x)].add(Edge(tube, (y, x-1)))
                        edges[(y, x-1)].add(Edge(tube, (y-1, x)))
                    case '7':
                        edges[(y+1, x)].add(Edge(tube, (y, x-1)))
                        edges[(y, x-1)].add(Edge(tube, (y+1, x)))
                    case 'F':
                        edges[(y+1, x)].add(Edge(tube, (y, x+1)))
                        edges[(y, x+1)].add(Edge(tube, (y+1, x)))
                x += 2
            y += 2
    starts: set[Edge] = set()
    for i in [-1, +1]:
        if edges[(start[0][0]+i, start[0][1])]:
            starts.add(Edge(start, (start[0][0]+i, start[0][1])))
        if edges[(start[0][0], start[0][1]+i)]:
            starts.add(Edge(start, (start[0][0], start[0][1]+i)))
    return starts, edges, tubesMap

def printField(starts: set[Edge], tubesMap: dict[Node, str], nodes: Iterable[Node] = list(), nodeChar: str = 'o'):
    """
    Prints the field
    """

    tubeChar = {'|': '║', '-': '═', 'L': '╚', 'J': '╝', '7': '╗', 'F': '╔', 'S': '╬'}

    startNodes = [s.node for s in starts]
    allKeys = set(nodes).union(tubesMap.keys()).union(startNodes)
    minX = min([x for _, x in allKeys])
    minY = min([y for y, _ in allKeys])
    maxX = max([x for _, x in allKeys])
    maxY = max([y for y, _ in allKeys])
    for y in range(minY, maxY+1):
        for x in range(minX, maxX+1):
            if (y, x) in tubesMap:
                c = tubesMap[(y, x)]
                print(tubeChar[c] if c in tubeChar else c, end='')
            elif (y, x) in startNodes:
                print('S', end='')
            elif (y, x) in nodes:
                print(nodeChar, end='')
            else:
                print('.', end='')
        print()

In [2]:
starts, edges, tubesMap = readInput('test3.txt')

print('Field (tubes and nodes):')
printField(starts, tubesMap, edges)


def next(current: Node, edges: dict[Node, set[Edge]], visited: set[Node]) -> Edge:
    for e in edges[current]:
        if e.node not in visited:
            return e
    return None

def cycle(starts, edges) -> list[Edge]:
    for start in starts:
        current: Edge = start
        visited: set[Node] = set()
        visitedEdges: list[Edge] = list()
        while current is not None:
            visitedEdges.append(current)
            visited.add(current.node)
            current = next(current.node, edges, visited)
        
        if sum([1 if s.node in visited else 0 for s in starts]) == 2:
            return visitedEdges
path = cycle(starts, edges)

print("\nCycle (tubes only):")
printField(set(), {(p.tube[0][0]//2, p.tube[0][1]//2):p.tube[1] for p in path})

print(len(path)//2)


Field (tubes and nodes):
.........o...............................
.╔o╔o╗.╔S╬o╔o╗.╔o╗.╔o╗.╔o╗.╔o╗.╔o═o═o═o╗.
.o.o.o.o.S.o.o.o.o.o.o.o.o.o.o.o.......o.
.╚o║.╚o╝.║.║.║.║.║.║.║.║.║.║.║.║.╔o═o═o╝.
...o.....o.o.o.o.o.o.o.o.o.o.o.o.o.......
.╔o╚o═o╗.╚o╝.╚o╝.║.║.║.║.║.║.╚o╝.╚o═o╗o╗.
.o.....o.........o.o.o.o.o.o.........o.o.
.╔o═o═o╝.╔o═o═o╗.║.║.╚o╝.╚o╝o╗.╔o╗.╔o╝o═o
.o.......o.....o.o.o...o.o...o.o.o.o.o...
.╚o═o═o═o╝.╔o═o╝.╚o╝...║.║o═o╔o╝.╚o╝o╝o╗.
.o...o.....o...........o.o...o...o.o...o.
.║.╔o║.╔o═o╝.╔o═o═o═o╗.╔o╗o═o╚o╗.╚o║o╗.║.
.o.o.o.o.....o.......o.o.o.o.o.o...o.o.o.
.║.╔o╔o╝.╔o╗.╚o╗.╔o═o╝.╔o╗.║o╝.╚o═o═o═o╗.
.o.o.o...o.o...o.o.....o.o.o...........o.
o╗o═o╚o═o╝.╚o╗.║.║.╔o╗.║.╚o╗.╔o═o╗.╔o╗.║.
.o...o...o...o.o.o.o.o.o...o.o...o.o.o.o.
.╚o..╚o╗.╚o╔o╝.║.║.║.║.║.╔o╝.╚o╗.║.║.╚o╝.
.o...o.o.o.o...o.o.o.o.o.o.....o.o.o...o.
.╚o╗o╝.╚o╝.╚o═o╝.╚o╝.╚o╝.╚o═o═o╝.╚o╝...╚o
...o.....................................

Cycle (tubes only):
.╔╗╔╬╔╗╔╗╔╗╔╗╔╗╔═══╗
.║╚╝║║║║║║║║║║║║╔══╝
.╚═╗╚╝╚╝║║║║║║╚╝╚═╗.
╔══╝╔══╗║

## Part 2

In [3]:
starts, edges, tubesMap = readInput('test3.txt')

path = cycle(starts, edges)

allNodes = set([p.node for p in path]).union([p.tube[0] for p in path])

def floodOutside(path):
    visited: set[Node] = set()
    toVisit: set[Node] = set()

    allNodes = set([p.node for p in path]).union([p.tube[0] for p in path])


    maxL = max([n[0] for n in allNodes])
    maxC = max([n[1] for n in allNodes])

    for l in (0, maxL):
        for c in range(maxC+1):
            if (l, c) not in allNodes:
                toVisit.add((l, c))

    for c in (0, maxC):
        for l in range(maxL+1):
            if (l, c) not in allNodes:
                toVisit.add((l, c))
    
    outside = set()
    while toVisit:
        current = toVisit.pop()
        visited.add(current)
        if current not in allNodes:
            outside.add(current)
            for i in (-1, +1):
                n = (current[0]+i, current[1])
                if 0 <= n[0] <= maxL and n not in visited:
                    toVisit.add(n)
                n = (current[0], current[1]+i)
                if  0 <= n[1] <= maxC and n not in visited:
                    toVisit.add(n)

    return outside

print("Outside:")
outside = floodOutside(path)
cycleMap = {(p.tube[0][0], p.tube[0][1]):p.tube[1] for p in path}
cycleMap.update({(p.node[0], p.node[1]):'o' for p in path})
printField(set(), cycleMap, outside, '░')


tubeMap = {(p.tube[0][0]//2, p.tube[0][1]//2):p.tube[1] for p in path}
outside2 = set([(o[0]//2, o[1]//2) for o in outside])

print("\nOutside (tubes only):")
printField(set(), tubeMap, outside2, '░')

maxL = max(max([n[0] for n in tubeMap]), max([n[0] for n in outside2]))
maxC = max(max([n[1] for n in tubeMap]), max([n[1] for n in outside2]))

notInside = outside2.union(tubeMap.keys())

print((maxL+1)*(maxC+1) - len(notInside))
        
    

Outside:
░░╔o╗░╔o╬░╔o╗░╔o╗░╔o╗░╔o╗░╔o╗░╔o═o═o═o╗
░░o.o░o.o░o.o░o.o░o.o░o.o░o.o░o.......o
░░║.╚o╝.║░║.║░║.║░║.║░║.║░║.║░║.╔o═o═o╝
░░o.....o░o.o░o.o░o.o░o.o░o.o░o.o░░░░░░
░░╚o═o╗.╚o╝.╚o╝.║░║.║░║.║░║.╚o╝.╚o═o╗░░
░░░░░░o.........o░o.o░o.o░o.........o░░
╔o═o═o╝.╔o═o═o╗.║░║.╚o╝.╚o╝...╔o╗.╔o╝░░
o.......o░░░░░o.o░o...........o░o.o░░░░
╚o═o═o═o╝░╔o═o╝.╚o╝.........╔o╝░╚o╝░░░░
░░░░░░░░░░o.................o░░░░░░░░░░
░░░░░░╔o═o╝.╔o═o═o═o╗.......╚o╗░░░░░░░░
░░░░░░o.....o░░░░░░░o.........o░░░░░░░░
░░░░╔o╝.╔o╗.╚o╗░╔o═o╝.╔o╗.....╚o═o═o═o╗
░░░░o...o░o...o░o.....o░o.............o
░░░░╚o═o╝░╚o╗.║░║.╔o╗.║░╚o╗.╔o═o╗.╔o╗.║
░░░░░░░░░░░░o.o░o.o░o.o░░░o.o░░░o.o░o.o
░░░░░░░░░░╔o╝.║░║.║░║.║░╔o╝.╚o╗░║.║░╚o╝
░░░░░░░░░░o...o░o.o░o.o░o.....o░o.o░░░░
░░░░░░░░░░╚o═o╝░╚o╝░╚o╝░╚o═o═o╝░╚o╝░░░░

Outside (tubes only):
░╔╗╔╬╔╗╔╗╔╗╔╗╔╗╔═══╗
░║╚╝║║║║║║║║║║║║╔══╝
░╚═╗╚╝╚╝║║║║║║╚╝╚═╗░
╔══╝╔══╗║║╚╝╚╝.╔╗╔╝░
╚═══╝╔═╝╚╝....╔╝╚╝░░
░░░╔═╝╔═══╗...╚╗░░░░
░░╔╝╔╗╚╗╔═╝╔╗..╚═══╗
░░╚═╝╚╗║║╔╗║╚╗╔═╗╔╗║
░░░░░╔╝║║║║║╔╝╚╗║║╚╝
░░░░░╚═╝╚╝╚╝╚══╝╚╝░